<a href="https://colab.research.google.com/github/ryukkt62/Study/blob/main/LLM_Study/Korean_RAG_Pipeline_with_LangChain_%2B_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 패키지 설치
  - Retrieval Augmented Generation(RAG) pipeline 구성을 위해 필요한 패키지 설치

In [ ]:
!pip install -q langchain langchain-huggingface langchain-community langchain-core langchain-text-splitters bitsandbytes docx2txt langchain-chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. 문서 split 및 Chroma를 활용한 vector store 구성
  - 'intfloat/multilingual-e5-large-instruct'을 Embedding으로 활용

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=100
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large-instruct')


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma

collection_name = 'chroma_tax'

database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    collection_name=collection_name,
    persist_directory='./chroma_huggingface'
)

# 3. HuggingFace에서 LLM 다운로드
  - 작업 시점 기준 가장 괜찮은 성능을 내는 `'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct'` 활용

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

chat_model = HuggingFacePipeline.from_model_id(
    model_id='LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct',
    task='text-generation',
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        do_sample=False,
        repetition_penalty=1.03
    ),
    model_kwargs={'quantization_config': quantization_config}
)

tokenizer_config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
llm = ChatHuggingFace(llm=chat_model)

# 4. `create_stuff_documents_chain` 활용한 RAG pipeline 구현

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retriever = database.as_retriever(search_kwargs={"k": 1})



/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
query = "연봉 5천만원인 거주자의 소득세는?"

retrieved_docs = retriever.invoke(query)

In [ ]:
retrieved_docs

[Document(metadata={'source': './tax_with_markdown.docx'}, page_content='⑨ 거주자가 다음 각 호의 어느 하나에 해당하는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제(이하 “표준세액공제”라 한다)한다.<개정 2020. 12. 29.>\n\n1. 근로소득이 있는 거주자로서 제6항, 제52조제8항 및 「조세특례제한법」 제95조의2제2항에 따른 소득공제나 세액공제 신청을 하지 아니한 경우: 연 13만원\n\n2. 종합소득이 있는 거주자(근로소득이 있는 자는 제외한다)로서 「조세특례제한법」 제122조의3에 따른 세액공제 신청을 하지 아니한 경우: 다음 각 목의 구분에 따른 금액\n\n가. 제160조의5제3항에 따른 사업용계좌의 신고 등 대통령령으로 정하는 요건에 해당하는 사업자(이하 “성실사업자”라 한다)의 경우: 연 12만원\n\n나. 가목 외의 경우: 연 7만원\n\n⑩ 제1항부터 제9항까지의 규정에 따른 공제를 “특별세액공제”라 한다.\n\n⑪ 특별세액공제에 관하여 그밖에 필요한 사항은 대통령령으로 정한다.\n\n[본조신설 2014. 1. 1.]\n\n\n\n제59조의5(세액의 감면) ① 종합소득금액 중 다음 각 호의 어느 하나의 소득이 있을 때에는 종합소득 산출세액에서 그 세액에 해당 근로소득금액 또는 사업소득금액이 종합소득금액에서 차지하는 비율을 곱하여 계산한 금액 상당액을 감면한다. <개정 2013. 1. 1.>')]

In [ ]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
ai_message = retrieval_chain.invoke({"input": query})

In [ ]:
ai_message

{'input': '연봉 5천만원인 거주자의 소득세는?',
 'context': [Document(metadata={'source': './tax_with_markdown.docx'}, page_content='⑨ 거주자가 다음 각 호의 어느 하나에 해당하는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제(이하 “표준세액공제”라 한다)한다.<개정 2020. 12. 29.>\n\n1. 근로소득이 있는 거주자로서 제6항, 제52조제8항 및 「조세특례제한법」 제95조의2제2항에 따른 소득공제나 세액공제 신청을 하지 아니한 경우: 연 13만원\n\n2. 종합소득이 있는 거주자(근로소득이 있는 자는 제외한다)로서 「조세특례제한법」 제122조의3에 따른 세액공제 신청을 하지 아니한 경우: 다음 각 목의 구분에 따른 금액\n\n가. 제160조의5제3항에 따른 사업용계좌의 신고 등 대통령령으로 정하는 요건에 해당하는 사업자(이하 “성실사업자”라 한다)의 경우: 연 12만원\n\n나. 가목 외의 경우: 연 7만원\n\n⑩ 제1항부터 제9항까지의 규정에 따른 공제를 “특별세액공제”라 한다.\n\n⑪ 특별세액공제에 관하여 그밖에 필요한 사항은 대통령령으로 정한다.\n\n[본조신설 2014. 1. 1.]\n\n\n\n제59조의5(세액의 감면) ① 종합소득금액 중 다음 각 호의 어느 하나의 소득이 있을 때에는 종합소득 산출세액에서 그 세액에 해당 근로소득금액 또는 사업소득금액이 종합소득금액에서 차지하는 비율을 곱하여 계산한 금액 상당액을 감면한다. <개정 2013. 1. 1.>')],
 'answer': '[|system|]Answer any use questions based solely on the context below:\n\n<context>\n⑨ 거주자가 다음 각 호의 어느 하나에 해당하는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제(이하 “표준세액공제”라 한다)한다.<개정 2020. 12. 29.>\n\n1. 근로소득이 있는 거주자로서

In [ ]:
test_ai_message = llm.invoke('인프런에는 어떤 강의가 있나요?')

In [ ]:
test_ai_message

AIMessage(content='[|system|][|endofturn|]\n[|user|]인프런에는 어떤 강의가 있나요?\n[|assistant|]인프런(inflearn)은 한국에서 운영되는 온라인 교육 플랫폼으로, 다양한 프로그래밍 언어와 관련 기술을 배울 수 있는 강의를 제공합니다. 다음은 인프런에서 찾을 수 있는 주요 강의 분야입니다:\n\n1. **프로그래밍 언어**:\n   - Python\n   - Java\n   - C/C++\n   - JavaScript\n   - Go\n   - Ruby\n\n2. **웹 개발**:\n   - HTML/CSS\n   - JavaScript (프론트엔드)\n   - React.js\n   - Node.js\n   - Django\n   - Spring Framework\n\n3. **데이터베이스**:\n   - MySQL\n   - PostgreSQL\n   - MongoDB\n   - Oracle Database\n\n4. **서버 및 네트워크**:\n   - Linux\n   - Docker\n   - Kubernetes\n   - AWS\n   - 네트워크 기초\n\n5. **데이터 과학 및 머신러닝**:\n   - R 프로그래밍\n   - TensorFlow\n   - PyTorch\n   - 데이터 분석\n\n6. **버전 관리 및 협업 도구**:\n   - Git\n   - GitHub\n   - Bitbucket\n\n7. **테스트 및 품질 관리**:\n   - JUnit\n   - Selenium\n   - TestNG\n\n8. **기타 기술 및 도구**:\n   - SQLAlchemy\n   - Flask\n   - Django REST framework\n   - Flask-RESTful\n\n이 외에도 최신 트렌드에 맞춘 다양한 주제의 강의들이 지속적으로 업데이트되고 있습니다. 인프런에서는 초보자부터 고급 개발자까지 다양한 수준의 학습 자료를 제공하고 있으니, 관심 있는 분야의 강의를 